In [95]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os
import logging
import sys
import time
load_dotenv()

True

In [96]:
artist_names = pd.read_csv('/Users/anastasiahimic/hse_python_project_2-1/data_scrapping/files/spotify_artists.csv')['Rank']   

In [97]:
artist_names

0                  Drake
1           Taylor Swift
2              Bad Bunny
3             The Weeknd
4          Justin Bieber
              ...       
2995        Gilberto Gil
2996                Roar
2997             Redimi2
2998         Jessie Ware
2999    Marco Barrientos
Name: Rank, Length: 3000, dtype: object

In [98]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True,  # перезаписывает любые ранние настройки/хендлеры
)

logging.info("Загружены переменные из .env")

logger = logging.getLogger("spotify")


2025-11-12 12:11:57,192 INFO root Загружены переменные из .env


Чтобы дальше все работало, необходимо в корне создать файл .env с client_id и client_secret

In [ ]:
client_id = os.getenv('SPOTIPY_CLIENT_ID') or os.getenv('client_id_sp')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET') or os.getenv('client_secret_sp')

In [100]:
if not client_id or not client_secret:
    logging.error("Отсутствуют client_id или client_secret. Установите SPOTIPY_CLIENT_ID и SPOTIPY_CLIENT_SECRET или client_id_sp/client_secret_sp в .env")
    raise RuntimeError("No client_id or client_secret.")
try:
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    logging.info("Аутентификация (Client Credentials) успешно выполнена")
except Exception as e:
    logging.error(f"Ошибка при аутентификации (Client Credentials): {e}")
    raise

2025-11-12 12:11:57,212 INFO root Аутентификация (Client Credentials) успешно выполнена


Функция проходит по списку имён, ищет артиста в Spotify для каждого имени и формирует словарь имя - artist_id. На каждом шаге считает, сколько имён обработано, и логирует успехи и случаи «не найдено». Между запросами есть пауза 0.2 c, чтобы не упираться в лимиты API. В конце логируется сводка с числом входов и числом записей в словаре и возвращается результат в виде dict.

In [101]:
def name_to_id(sp, names):
    logger.info("name_to_id: started") # фиксируем старт выполнения функции
    out = {}
    total = 0
    for n in pd.Series(names).dropna().astype(str).str.strip():
        total += 1
        r = sp.search(q=f"artist:{n}", type="artist", limit=1)  # Поиск в Spotify
        items = r.get("artists", {}).get("items", [])
        artist_id = items[0]["id"] if items else ""
        out[n] = artist_id
        if artist_id:
            logger.info(f"name_to_id: '{n}' -> id={artist_id}")
        else:
            logger.warning(f"name_to_id: '{n}' not found (empty id)")
        time.sleep(0.2) # потому что спотик козявка и баннит меня 3 раз(
    logger.info(f"name_to_id: completed; unique input={total}, mapped={len(out)}")
    return out

Функция батчами запрашивает детали артистов по их Spotify ID и собирает словарь {id - объект артиста}. Логирует старт, размер входа и для каждого батча показывает, сколько id запросили и сколько валидных записей получили. Цикл идёт по срезам длиной bs, после каждого запроса стоит пауза 0.8 c, чтобы не упираться в лимиты API. В конце логируется общее число полученных артистов и возвращается итоговый словарь.

In [102]:
def fetch_artists(sp, ids, bs=50):
    logger.info(f"fetch_artists: started; ids_in={len(ids)}, batch_size={bs}")
    m = {}
    total_batches = (len(ids) + bs - 1) // bs
    for bi, start in enumerate(range(0, len(ids), bs), start=1):
        batch = ids[start:start+bs]
        resp = sp.artists(batch).get("artists", [])
        got = {a.get("id"): a for a in resp if a and a.get("id")}
        m.update(got)
        logger.info(f"fetch_artists: batch {bi}/{total_batches}; requested={len(batch)}, received={len(got)}")
        time.sleep(0.8) # потому что спотик козявка и баннит меня 3 раз(
    logger.info(f"fetch_artists: completed; total_received={len(m)}")
    return m

Функция собирает таблицу по именам артистов: сначала маппит имена в Spotify ID, затем батчем подтягивает полные объекты артистов и формирует список строк для DataFrame. Для каждого имени кладутся поля: artist, artist_id, followers.total, genres (склеенные через запятую), popularity и type. Логи фиксируют старт, каждую собранную строку в DEBUG и итоговое число строк, после чего возвращается pandas DataFrame.

In [103]:
def build_table(sp, names):
    logger.info("build_table: started")
    m = name_to_id(sp, names)
    full = fetch_artists(sp, list(m.values()))
    rows = []
    for name, aid in m.items():
        a = full.get(aid, {}) if aid else {}
        row = {
            "artist": name,
            "artist_id": aid,
            "followers": (a.get("followers") or {}).get("total"),
            "genres": ", ".join(a.get("genres", []) or []),
            "popularity": a.get("popularity"),
            "type": a.get("type"),
        }
        rows.append(row)
        logger.debug(f"build_table: row {row}")
    logger.info(f"build_table: completed; rows={len(rows)}")
    return pd.DataFrame(rows)

Если хотите получить все, уберите .head()

In [104]:
df = build_table(sp, artist_names.head())  # пример 

2025-11-12 12:11:57,234 INFO spotify build_table: started
2025-11-12 12:11:57,235 INFO spotify name_to_id: started
2025-11-12 12:11:57,578 INFO spotify name_to_id: 'Drake' -> id=3TVXtAsR1Inumwj472S9r4
2025-11-12 12:11:58,002 INFO spotify name_to_id: 'Taylor Swift' -> id=06HL4z0CvFAxyc27GXpf02
2025-11-12 12:11:58,451 INFO spotify name_to_id: 'Bad Bunny' -> id=4q3ewBCX7sLwd24euuV69X
2025-11-12 12:11:58,951 INFO spotify name_to_id: 'The Weeknd' -> id=1Xyo4u8uXC1ZmMpatF05PJ
2025-11-12 12:11:59,352 INFO spotify name_to_id: 'Justin Bieber' -> id=1uNFoZAHBGtllmzznpCI3s
2025-11-12 12:11:59,556 INFO spotify name_to_id: completed; unique input=5, mapped=5
2025-11-12 12:11:59,556 INFO spotify fetch_artists: started; ids_in=5, batch_size=50
2025-11-12 12:11:59,715 INFO spotify fetch_artists: batch 1/1; requested=5, received=5
2025-11-12 12:12:00,521 INFO spotify fetch_artists: completed; total_received=5
2025-11-12 12:12:00,521 INFO spotify build_table: completed; rows=5


Уберите # и тогда создастся csv

In [105]:
#df.to_csv(os.path.join(os.getcwd(), "artists_stats.csv"), index=False, encoding="utf-8")